In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [122]:
sumo = {}
sumo_files = os.listdir('data_experiment1_sumo')
for file in sumo_files:
    sumo_df = pd.read_csv(f'data_experiment1_sumo/{file}')
    sumo[file.split(".")[0]] = sumo_df
print(sumo.keys())

dict_keys(['veh2_normal', 'veh3_aggressive', 'veh4_aggressive', 'veh2_aggressive', 'veh4_normal', 'veh1_aggressive', 'veh0_aggressive', 'veh0_normal', 'veh3_normal', 'veh1_normal'])


In [123]:
carla = {}
carla_files = os.listdir('data_experiment1_carla')
for file in carla_files:
    carla_df = pd.read_csv(f'data_experiment1_carla/{file}')
    carla_df.rename(columns={'latitude': 'y_pos', 'longitude': 'x_pos'}, inplace=True)
    carla[file.split('.')[0]] = carla_df[carla_df['timestamp'] > 0]
print(carla.keys())

dict_keys(['veh2_normal', 'veh3_aggressive', 'veh4_aggressive', 'veh2_aggressive', 'veh4_normal', 'veh1_aggressive', 'veh0_aggressive', 'veh0_normal', 'veh3_normal', 'veh1_normal'])


In [125]:
carla['veh0_aggressive'].head()

,timestamp,acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z,compass,y_pos,x_pos
1,0.1,0.010970,0.000000,9.811547,1.798589e-07,-8.505239e-09,1.390007e-09,1.532124,0.000269,0.000317
2,0.2,0.010969,0.000000,9.810353,9.482394e-07,2.249508e-07,-1.543382e-09,1.532124,0.000269,0.000317
3,0.3,0.010968,0.000000,9.809747,1.129561e-06,1.361366e-07,-1.802021e-09,1.532124,0.000269,0.000317
4,0.4,4.402090,0.312138,9.566172,1.647623e-02,-7.297179e-02,5.142259e-02,1.675613,0.000270,0.000317
5,0.5,8.509295,-0.180517,9.757805,-7.991903e-03,-7.402906e-04,1.602896e-02,1.730314,0.000271,0.000317


In [126]:
sumo['veh0_aggressive'].head()

,timestamp,x_pos,y_pos,speed,speed_x,speed_y,acc,acc_x,acc_y,angle,acc_diff,gyro_z
0,0.1,305.751528,218.361287,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.590300,0.000000,0.000000
1,0.2,305.751546,218.363007,0.017195,0.017194,0.000177,0.171948,0.171939,0.001772,0.590499,0.171948,0.000003
2,0.3,305.751595,218.367904,0.048955,0.048953,0.000505,0.317604,0.317587,0.003276,0.591066,0.317604,0.000000
3,0.4,305.751701,218.378423,0.105161,0.105155,0.001087,0.562057,0.562027,0.005810,0.592283,0.244453,0.000021
4,0.5,305.751901,218.398237,0.198090,0.198080,0.002056,0.929295,0.929245,0.009643,0.594576,0.367238,0.000040


The time series may have different lengths because of the routing and simulation of each simulator.

In [127]:
for (sumo_k, sumo_v), (carla_k, carla_v) in zip(sumo.items(), carla.items()):
    print(f'Sumo {sumo_k} size: {len(sumo_v)}. Carla {sumo_k} size: {len(carla_v)}')


Sumo veh2_normal size: 7818. Carla veh2_normal size: 7200
Sumo veh3_aggressive size: 8416. Carla veh3_aggressive size: 7858
Sumo veh4_aggressive size: 7306. Carla veh4_aggressive size: 7076
Sumo veh2_aggressive size: 7135. Carla veh2_aggressive size: 6741
Sumo veh4_normal size: 8001. Carla veh4_normal size: 7521
Sumo veh1_aggressive size: 8239. Carla veh1_aggressive size: 7692
Sumo veh0_aggressive size: 7328. Carla veh0_aggressive size: 7013
Sumo veh0_normal size: 8003. Carla veh0_normal size: 7482
Sumo veh3_normal size: 9223. Carla veh3_normal size: 8450
Sumo veh1_normal size: 8967. Carla veh1_normal size: 8154


Note que gyro_z não é obtido diretamente pelo SUMO, aqui está sendo realizada uma aproximação a partir da variação nos ângulos

In [142]:
def plot_df(sumo_df, carla_df, idx, save_path = None):
    # Create subplots
    fig, axes = plt.subplots(3, 2, figsize=(20, 20))
    axes = axes.flatten()

    lenght = min(len(sumo_df), len(carla_df))
    # sumo_df = sumo_df[:lenght]
    # carla_df = carla_df[:lenght]

    alpha = 0.6

    # Plot acceleration
    axes[0].plot(sumo_df['timestamp'], sumo_df['acc_x'], label='Sumo Acc X', alpha=alpha)
    axes[0].plot(carla_df['timestamp'], carla_df['acc_x'], label='Carla Acc X', alpha=alpha)
    axes[0].set_title("Acceleration X over Time")
    axes[0].set_ylabel("Acceleration (m/s²)")
    axes[0].legend()
    axes[0].grid(True)

    axes[1].plot(sumo_df['timestamp'], sumo_df['acc_y'], label='Sumo Acc Y', alpha=alpha)
    axes[1].plot(carla_df['timestamp'], carla_df['acc_y'], label='Carla Acc Y', alpha=alpha)
    axes[1].set_title("Acceleration Y over Time")
    axes[1].set_ylabel("Acceleration (m/s²)")
    axes[1].legend()
    axes[1].grid(True)

    # Plot gyroscope
    axes[2].plot(sumo_df['timestamp'], sumo_df['gyro_z'], label='Sumo Gyro Z', alpha=alpha)
    axes[2].plot(carla_df['timestamp'], carla_df['gyro_z'], label='Carla Gyro Z', alpha=alpha)
    axes[2].set_title("Gyroscope Z over Time")
    axes[2].set_ylabel("Angular Velocity (rad/s)")
    axes[2].legend()
    axes[2].grid(True)

    # Plot angle
    axes[3].plot(sumo_df['timestamp'], sumo_df['angle'], label='Sumo Angle', alpha=alpha)
    axes[3].plot(carla_df['timestamp'], carla_df['compass'], label='Carla Angle', alpha=alpha)
    axes[3].set_title("Angle over Time")
    axes[3].set_ylabel("Angle (rad)")
    axes[3].legend()
    axes[3].grid(True)

    # Plot position
    scatter = axes[4].scatter(sumo_df['x_pos'], sumo_df['y_pos'], c=sumo_df['timestamp'], cmap='viridis', s=10)
    axes[4].set_title("Sumo Position over Time")
    axes[4].set_xlabel("X position")
    axes[4].set_ylabel("Y position")
    axes[4].grid(True)
    cbar = fig.colorbar(scatter, ax=axes[4])
    cbar.set_label("Time (s)")

    scatter = axes[5].scatter(carla_df['x_pos'], carla_df['y_pos'], c=carla_df['timestamp'], cmap='viridis', s=10)
    axes[5].set_title("Carla Position over Time")
    axes[5].set_xlabel("X position")
    axes[5].set_ylabel("Y position")
    axes[5].grid(True)
    cbar = fig.colorbar(scatter, ax=axes[5])
    cbar.set_label("Time (s)")

    # Set the main title
    fig.tight_layout(rect=[0, 0.03, 1, 0.96])  # Leave space at the top for suptitle
    fig.suptitle(f'Comparison of SUMO and CARLA Data for {idx}', fontsize=18)
    
    if save_path:
        plt.savefig(save_path)
        plt.close(fig)
    else:
        plt.show()


In [143]:
os.makedirs('plots', exist_ok=True)
for veh in sumo.keys():
    plot_df(sumo[veh], carla[veh], veh, save_path=f'plots/{veh}.png')